In [1]:
import numpy as np
import pandas as pd
import requests 
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

from urllib.request import urlopen
import urllib

#from tqdm import tqdm

from selenium import webdriver
import openpyxl
import time
import math
from collections import Counter

import re
import json

from tqdm.notebook import tqdm

In [2]:
def url_split(url):
    main_,para_1 = url.split("?")
    para_2 = para_1.split("&")
    para_3 = [d.replace("=",":").split(":") for d in para_2]
    params_ = {}
    for key,value in para_3:
        params_[key] = value
    return main_,params_

In [3]:
ua = UserAgent()

# Main page 

In [4]:
def Main_page_SSG(Key_,Page_):
    ua = UserAgent()
    header = {'User-Agent':ua.chrome,
              "referer":"http://www.ssg.com/"}
    page_ = Page_
    key_ = Key_

    url = "http://www.ssg.com/search.ssg?target=all&query={}&sort=cnt&page={}".format(key_,page_)
    html = requests.get(url.format(key_),headers=header).text
    soup = BeautifulSoup(html,"html.parser")
    url_base_ = "http://www.ssg.com"
    name_head,url_head,reveiw_count_1 = [],[],[]
    for tag in soup.select("li"):
        try:
            name_head.append(tag.find("div",class_="cunit_info").find("div",class_="cunit_md notranslate").find("a").find("em",class_="tx_ko").text)
            url_head.append(tag.find("div",class_="cunit_info").find("div",class_="cunit_md notranslate").find("a")["href"])
            reveiw_count_1.append(int(tag.find("div",class_="cunit_app").find("span",class_="rate_tx").find("em").text.replace(",","")))
    
        except:
            pass
    Head_df = pd.DataFrame([name_head,reveiw_count_1,url_head]).T.rename(columns={0:"Name",1:"Review_count",2:"Url"})
    return Head_df

In [5]:
def Main_page_concat(Total_p,Key_):
    tq = tqdm(range(Total_p))
    head_all = pd.DataFrame()
    for tp in tq:
        pa_ = tp + 1
        candi_head =  Main_page_SSG(Key_,pa_)
        head_all = pd.concat([head_all,candi_head]).reset_index(drop=True)
    return head_all

In [6]:
head_df = Main_page_concat(3,"헤어드라이기")
head_df

  0%|          | 0/3 [00:00<?, ?it/s]

,Name,Review_count,Url
0,"플라 이지랩 자동 봉고데기 및 드라이기, 고데기, 두피케어~HIT 헤어스타일러 제안",3339,/item/dealItemView.ssg?itemId=1000065874177&si...
1,★[브랜드위크 20% 쿠폰] JMW 항공모터 드라이기/고데기 특가,2690,/item/dealItemView.ssg?itemId=1000039917682&si...
2,★[브랜드위크 20% 쿠폰] JMW 프리미엄 항공모터 드라이기/고데기 특가,2690,/item/dealItemView.ssg?itemId=1000036476134&si...
3,다이슨코리아 신제품 슈퍼소닉/에어랩 갓스타일의 완성,1544,/item/dealItemView.ssg?itemId=1000040673169&si...
4,★15%쿠폰할인BLDC항공모터드라이기&고데기모음전,1359,/item/dealItemView.ssg?itemId=1000039492458&si...
...,...,...,...
235,D590PK - 1600W 접이식 레트로 프로페셔널 음이온 헤어 드라이기 [벽걸이용...,None,/item/itemView.ssg?itemId=1000010770411&siteNo...
236,5224K - 1700W 코드키퍼 헤어 드라이기,None,/item/itemView.ssg?itemId=0000010701502&siteNo...
237,어느 공간에서도 예쁘게~ 컬러 맛집! 레이트 접이식 드라이기 F600 모음전,None,/item/dealItemView.ssg?itemId=1000063156160&si...
238,[S머니 적립] 오아 이지고데기 뿌리 볼륨 앞머리 헤어 웨이브 판 고데기,None,/item/itemView.ssg?itemId=1000054230181&siteNo...


# Review_page

In [31]:
def Review_URL(head_df,target_row):
    """
    head_df = main page information
    target_row = head_df's target product
    page_2 = target product's review pages
    """
    url_base_ = 'http://www.ssg.com'
    Target_name = head_df["Name"][target_row]
    target_url = head_df["Url"][target_row]#.encode('utf-8')
    Target_max_p = head_df["Review_count"][target_row]

    ## Request HTML
    header = {'User-Agent':ua.chrome,
              "referer":"http://www.ssg.com/"}

    html = requests.get(url_base_ + target_url,headers=header).text
    soup = BeautifulSoup(html,"html.parser")

    ## Review_page Params data

    actu_num = 0
    for tag in soup.select(".chd_lst_choice "):
        try:
            actu_num = tag.find("li",class_="item")["data-deal-option"]
        except:
            pass

    dic_ = {}
    for m in [i.split("=") for i in target_url.split("?")[1].split("&")]:
        dic_[m[0]] = m[1]
    dic_["itemId"] = actu_num

    ## Review_page Request

    url_re_base = "http://www.ssg.com/item/ajaxItemCommentList.ssg?itemId={}&siteNo={}&page={}&pageSize={}"
    item_id = dic_["itemId"]
    site_no = dic_["siteNo"]
    page_sz = 20
    Url_review = url_re_base.format(item_id,site_no,"{}",page_sz)
    return Url_review,Target_name,Target_max_p

In [32]:
url_review,target_name,max_page = Review_URL(head_df,1)
url_review

'http://www.ssg.com/item/ajaxItemCommentList.ssg?itemId=1000031479690&siteNo=6004&page={}&pageSize=20'

In [33]:
target_name

'★[브랜드위크 20% 쿠폰] JMW 항공모터 드라이기/고데기 특가'

In [34]:
max_page//10

269

In [35]:
url_review

'http://www.ssg.com/item/ajaxItemCommentList.ssg?itemId=1000031479690&siteNo=6004&page={}&pageSize=20'

In [36]:
def Review_page_ex(url_review,target_name):
    header = {'User-Agent':ua.chrome,
              "referer":"http://www.ssg.com/"}

    html = requests.get(url_review,headers=header).text
    review_n = len(json.loads(html)["pageDto"]["resultList"])
    review_1 = json.loads(html)["pageDto"]["resultList"]
    display(html)
    data_all = []
    for re_i in range(len(json.loads(html)["pageDto"]["resultList"])):
        date_ = json.loads(html)["pageDto"]["resultList"][re_i]["wrtDt"]
        id_ = json.loads(html)["pageDto"]["resultList"][re_i]["mbrLoginId"]
        item_ = target_name
        score_ = json.loads(html)["pageDto"]["resultList"][re_i]["recomEvalScr"]
        contents_ = json.loads(html)["pageDto"]["resultList"][re_i]["postngTitleNm"]
        liked_ = ""
        channel_ = "SSG"
        all_ = [date_,id_,item_,score_,contents_,liked_,channel_]
        data_all.append(all_)
    df_ = pd.DataFrame(data_all).rename(columns={0:"date",1:"id",2:"item",3:"score",4:"contents",5:"liked",6:"channel"})
    return df_

In [37]:
ur_ = 'http://www.ssg.com/item/ajaxItemCommentList.ssg?itemId=1000102149612&siteNo=6004&page=1&pageSize=20'
tn_ = '플라 이지랩 자동 봉고데기 및 드라이기, 고데기, 두피케어~HIT 헤어스타일러 제안'
Review_page_ex(ur_,tn_)

'{"bbsId":null,"postngId":null,"postngTitleNm":"","wrtDt":null,"wrtHms":null,"ansstYn":null,"bbsTypeCd":null,"postngCnttTypeCd":null,"postngTgtNo":null,"wrtpeDivCd":null,"wrtpeIdnfNo":null,"postngClsCd":null,"postngStatCd":null,"postngBrwsCnt":null,"postngRcmdCnt":null,"adminRcmdCnt":null,"bestPrioyRankg":null,"dispDisabRsnCd":null,"dispDisabRsnDtlCntt":null,"dispStatChngDts":null,"dispStatChngUserId":null,"itemId":null,"resMdUserId":null,"splVenId":null,"siteNo":null,"infloSiteNo":null,"dvicDivCd":null,"ansstPostngId":null,"postStrtDt":null,"postStrtHms":null,"postEndDt":null,"postEndHms":null,"upmostPostngYn":null,"rcvGrpApnmtYn":null,"postngProcStatCd":null,"imgBeingYn":null,"vodBeingYn":null,"newrtgMarkDcnt":null,"dispCtgId":null,"ordNo":null,"ordItemSeq":null,"uitemId":null,"inqSalestrNo":null,"recomEvalScr":null,"recomAssiScr":null,"spclzSuvyCnt":null,"purchYn":null,"offerPayYn":null,"ansWordTypeCd":null,"sortOrdr":null,"fixSortOrdr":null,"regpeId":null,"regDts":null,"modpeId":nu

""


In [19]:
def Crawling_SSG_all(head_df,TA_):
    Url_review,target_name,max_page = Review_URL(head_df,TA_)
    MaxPage = max_page//10
    tq = tqdm(range(MaxPage))

    review_all = pd.DataFrame()

    for mp in tq:
        url_review =Url_review.format(mp+1)
        
        review_candi = Review_page_ex(url_review,target_name)
        display(url_review)
        display(target_name)
        display(review_candi)
        review_all = pd.concat([review_all,review_candi]).reset_index(drop=True)
        
    return review_all,target_name

In [20]:
out_,NA_ = Crawling_SSG_all(head_df,0)
out_

  0%|          | 0/333 [00:00<?, ?it/s]

'http://www.ssg.com/item/ajaxItemCommentList.ssg?itemId=1000102149612&siteNo=6004&page=1&pageSize=20'

'플라 이지랩 자동 봉고데기 및 드라이기, 고데기, 두피케어~HIT 헤어스타일러 제안'

""


'http://www.ssg.com/item/ajaxItemCommentList.ssg?itemId=1000102149612&siteNo=6004&page=2&pageSize=20'

...

""


'http://www.ssg.com/item/ajaxItemCommentList.ssg?itemId=1000102149612&siteNo=6004&page=9&pageSize=20'

'플라 이지랩 자동 봉고데기 및 드라이기, 고데기, 두피케어~HIT 헤어스타일러 제안'

""


KeyboardInterrupt: 

In [15]:
def SSG_Save_local(path_,head_df):
    for i_ in range(len(head_df)):
        print("In {} Pro From {}".format(i_+1,len(head_df)+1))
        print()
        print(head_df["Name"][i_])
        out_,NA_ = Crawling_SSG_all(head_df,i_)
        NA_ = NA_.replace("/","")
        out_.to_excel(path_.format(NA_))

In [16]:
path_ = r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\hair\shopping\SSG\{}.xlsx"
SSG_Save_local(path_,head_df)

In 1 Pro From 161

플라 이지랩 자동 봉고데기 및 드라이기, 고데기, 두피케어~HIT 헤어스타일러 제안


  0%|          | 0/297 [00:00<?, ?it/s]

In 2 Pro From 161

★[브랜드위크 20% 쿠폰] JMW 항공모터 드라이기/고데기 특가


  0%|          | 0/251 [00:00<?, ?it/s]

In 3 Pro From 161

★[브랜드위크 20% 쿠폰] JMW 프리미엄 항공모터 드라이기/고데기 특가


...

|          | 0/0 [00:00<?, ?it/s]

In 159 Pro From 161

어느 공간에서도 예쁘게~ 컬러 맛집! 레이트 접이식 드라이기 F600 모음전


|          | 0/0 [00:00<?, ?it/s]

In 160 Pro From 161

헤어퀵드라이장갑1P


|          | 0/0 [00:00<?, ?it/s]

In [ ]:
import numpy as np
import pandas as pd
import requests 
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

from urllib.request import urlopen
import urllib

#from tqdm import tqdm

from selenium import webdriver
import openpyxl
import time
import math
from collections import Counter

import re
import json

from tqdm.notebook import tqdm

def url_split(url):
    main_,para_1 = url.split("?")
    para_2 = para_1.split("&")
    para_3 = [d.replace("=",":").split(":") for d in para_2]
    params_ = {}
    for key,value in para_3:
        params_[key] = value
    return main_,params_

ua = UserAgent()

# Main page 

def Main_page_SSG(Key_,Page_):
    ua = UserAgent()
    header = {'User-Agent':ua.chrome,
              "referer":"http://www.ssg.com/"}
    page_ = Page_
    key_ = Key_

    url = "http://www.ssg.com/search.ssg?target=all&query={}&sort=cnt&page={}".format(key_,page_)
    html = requests.get(url.format(key_),headers=header).text
    soup = BeautifulSoup(html,"html.parser")
    url_base_ = "http://www.ssg.com"
    name_head,url_head,reveiw_count_1 = [],[],[]
    for tag in soup.select("li"):
        try:
            name_head.append(tag.find("div",class_="cunit_info").find("div",class_="cunit_md notranslate").find("a").find("em",class_="tx_ko").text)
            url_head.append(tag.find("div",class_="cunit_info").find("div",class_="cunit_md notranslate").find("a")["href"])
            reveiw_count_1.append(int(tag.find("div",class_="cunit_app").find("span",class_="rate_tx").find("em").text.replace(",","")))
    
        except:
            pass
    Head_df = pd.DataFrame([name_head,reveiw_count_1,url_head]).T.rename(columns={0:"Name",1:"Review_count",2:"Url"})
    return Head_df

def Main_page_concat(Total_p,Key_):
    tq = tqdm(range(Total_p))
    head_all = pd.DataFrame()
    for tp in tq:
        pa_ = tp + 1
        candi_head =  Main_page_SSG(Key_,pa_)
        head_all = pd.concat([head_all,candi_head]).reset_index(drop=True)
    return head_all

head_df = Main_page_concat(3,"헤어드라이기")
head_df

# Review_page

def Review_URL(head_df,target_row):
    """
    head_df = main page information
    target_row = head_df's target product
    page_2 = target product's review pages
    """
    url_base_ = 'http://www.ssg.com'
    Target_name = head_df["Name"][target_row]
    target_url = head_df["Url"][target_row]#.encode('utf-8')
    Target_max_p = head_df["Review_count"][target_row]

    ## Request HTML
    header = {'User-Agent':ua.chrome,
              "referer":"http://www.ssg.com/"}

    html = requests.get(url_base_ + target_url,headers=header).text
    soup = BeautifulSoup(html,"html.parser")

    ## Review_page Params data

    actu_num = 0
    for tag in soup.select(".chd_lst_choice "):
        try:
            actu_num = tag.find("li",class_="item")["data-deal-option"]
        except:
            pass

    dic_ = {}
    for m in [i.split("=") for i in target_url.split("?")[1].split("&")]:
        dic_[m[0]] = m[1]
    dic_["itemId"] = actu_num

    ## Review_page Request

    url_re_base = "http://www.ssg.com/item/ajaxItemCommentList.ssg?itemId={}&siteNo={}&page={}&pageSize={}"
    item_id = dic_["itemId"]
    site_no = dic_["siteNo"]
    page_sz = 20
    Url_review = url_re_base.format(item_id,site_no,"{}",page_sz)
    return Url_review,Target_name,Target_max_p



url_review,target_name,max_page = Review_URL(head_df,1)
url_review

target_name

max_page//10

url_review

def Review_page_ex(url_review,target_name):
    header = {'User-Agent':ua.chrome,
              "referer":"http://www.ssg.com/"}

    html = requests.get(url_review,headers=header).text
    review_n = len(json.loads(html)["pageDto"]["resultList"])
    review_1 = json.loads(html)["pageDto"]["resultList"]
    data_all = []
    for re_i in range(len(json.loads(html)["pageDto"]["resultList"])):
        date_ = json.loads(html)["pageDto"]["resultList"][re_i]["wrtDt"]
        id_ = json.loads(html)["pageDto"]["resultList"][re_i]["mbrLoginId"]
        item_ = target_name
        score_ = json.loads(html)["pageDto"]["resultList"][re_i]["recomEvalScr"]
        contents_ = json.loads(html)["pageDto"]["resultList"][re_i]["postngTitleNm"]
        liked_ = ""   
        all_ = [date_,id_,item_,score_,contents_,liked_]
        data_all.append(all_)
    df_ = pd.DataFrame(data_all).rename(columns={0:"date",1:"id",2:"item",3:"score",4:"contents",5:"liked"})
    return df_



def Crawling_SSG_all(head_df,TA_):
    Url_review,target_name,max_page = Review_URL(head_df,TA_)
    MaxPage = max_page//10
    tq = tqdm(range(MaxPage))

    review_all = pd.DataFrame()

    for mp in tq:
        url_review =Url_review.format(mp+1)

        review_candi = Review_page_ex(url_review,target_name)
        review_all = pd.concat([review_all,review_candi]).reset_index(drop=True)
    return review_all,target_name

out_,NA_ = Crawling_SSG_all(head_df,0)
out_



def SSG_Save_local(path_,head_df):
    for i_ in range(len(head_df)):
        print("In {} Pro From {}".format(i_+1,len(head_df)+1))
        print()
        print(head_df["Name"][i_])
        out_,NA_ = Crawling_SSG_all(head_df,i_)
        NA_ = NA_.replace("/","")
        out_.to_excel(path_.format(NA_))

path_ = r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\hair\shopping\SSG\{}.xlsx"
SSG_Save_local(path_,head_df)

